### Adjust Word Group Repeat With Youtube Link

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# file extention
file_ext = "Twogram_Threegram"

In [4]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [15]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/3-Adjust Word Group Repeat With Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
path_folder_file = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Talk Time/Result/2-Find Word Group Youtube Link/Twogram Threegram Youtube Link Selected_Manuel.xlsx"
sheet = "Sheet1"  # Sheet1

In [7]:
df_word_group_time_loc = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,search_string,start_time,end_time,video_url,video_id,count
0,önemli olan,497,499,https://www.youtube.com/watch?v=NTG7NZgQals&t=...,NTG7NZgQals,7
1,önemli olan,1726,1728,https://www.youtube.com/watch?v=vsDoBhQJ1hY&t=...,vsDoBhQJ1hY,7
2,önemli olan,6246,6248,https://www.youtube.com/watch?v=m3Clq7uulPg&t=...,m3Clq7uulPg,7
3,önemli olan,406,408,https://www.youtube.com/watch?v=kzEvSIem_n4&t=...,kzEvSIem_n4,7
4,önemli olan,6184,6186,https://www.youtube.com/watch?v=JV6ZUdw_lXQ&t=...,JV6ZUdw_lXQ,7
...,...,...,...,...,...,...
512,yardım ister misin,1947,1950,https://www.youtube.com/watch?v=SUfgeDos2Kw&t=...,SUfgeDos2Kw,1
513,orada mısın sen,6253,6255,https://www.youtube.com/watch?v=IDBLfpy15Jk&t=...,IDBLfpy15Jk,1
514,lütfen izin ver,511,513,https://www.youtube.com/watch?v=nTYDDNdrdZM&t=...,nTYDDNdrdZM,1
515,hayır sorun yok,781,783,https://www.youtube.com/watch?v=iSIdXpAd76I&t=...,iSIdXpAd76I,1


In [8]:
search_list = df_word_group_time_loc.iloc[:,0].unique()
len(search_list)

96

In [9]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["search_string","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,search_string,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [10]:
## All word group result convert to join result in one row
#df_result_repeat = pd.DataFrame()
#for word_group in search_list:
#    # for repeat 1
#    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
#    # for repeat 2
#    try:
#        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
#    except:
#        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
#    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
#    # repeat block
#    for i in range(len(df_word_group_search_repeat1)):        
#        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)
#
#    try:
#        df_link_default_var = df_link_default
#        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
#        if word_time_diff_var < 4:
#            word_time_diff_var2 = 3
#        else:
#            word_time_diff_var2 = word_time_diff_var+1.0
#
#        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
#    except:
#        pass
#
#    for j in range(3):
#        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
#        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
#
#df_result_repeat.reset_index(drop=True, inplace=True)
#df_result_repeat_join = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"])
#
#df_result_repeat.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time2.xlsx", index=False)
#df_result_repeat_join.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time_Join2.xlsx", index=False)

In [11]:
#df_result_repeat

In [12]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [13]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["search_string"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)


In [14]:
df_all_join_result

,video_id,start_time,end_time,search_string,count
0,"NTG7NZgQals,vsDoBhQJ1hY,m3Clq7uulPg,kzEvSIem_n...","497,1726,6246,406,6184,229,8797,497,5,497,5,497,5","499,1728,6248,408,6186,231,8800,499,8,499,8,499,8",önemli olan,7.0
1,"ncy57YkTtdU,cvsKxQ5nsZo,wH4L9NiHHIE,EC4gKcDZvu...","2514,62,6468,1395,353,1435,7343,6468,5,6468,5,...","2516,65,6470,1398,356,1437,7346,6470,8,6470,8,...",çok güzel değil,7.0
2,"zaHwaMoI314,tP3ou_3RGVg,iZjzaNo_vDk,LyvjoFQdsr...","286,6269,1228,131,240,2836,5735,1228,5,1228,5,...","288,6271,1230,133,242,2838,5737,1230,8,1230,8,...",çok güzel,7.0
3,"mpg2dM5DrGg,k1ni27c9C2k,oCsHNgql560,oCsHNgql56...","5691,535,5776,2315,209,558,3467,3467,5,3467,5,...","5692,537,5778,2317,212,559,3468,3468,8,3468,8,...",tamam mı,7.0
4,"GimA7_uhyWg,o0vdJUDd7hQ,od_ZPJBFYUA,J0CIPlClrb...","1635,356,6389,22,5280,2,32,22,5,22,5,22,5","1637,358,6391,23,5282,4,34,23,8,23,8,23,8",tam burada,7.0
...,...,...,...,...,...
91,"SUfgeDos2Kw,SUfgeDos2Kw,Q-8I-uMUMYA,SUfgeDos2K...","1947,1947,5,1947,5,1947,5","1950,1950,8,1950,8,1950,8",yardım ister misin,1.0
92,"IDBLfpy15Jk,IDBLfpy15Jk,Q-8I-uMUMYA,IDBLfpy15J...","6253,6253,5,6253,5,6253,5","6255,6255,8,6255,8,6255,8",orada mısın sen,1.0
93,"nTYDDNdrdZM,nTYDDNdrdZM,Q-8I-uMUMYA,nTYDDNdrdZ...","511,511,5,511,5,511,5","513,513,8,513,8,513,8",lütfen izin ver,1.0
94,"iSIdXpAd76I,iSIdXpAd76I,Q-8I-uMUMYA,iSIdXpAd76...","781,781,5,781,5,781,5","783,783,8,783,8,783,8",hayır sorun yok,1.0


#### Copy Move And Delete

In [16]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx")
output_file

['Turkish_200_Word_Youtube_Talk_Time_Join_Twogram_Threegram.xlsx']

In [17]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [18]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Temp

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("Twogram Threegram Youtube Link Selected.xlsx", sheet_name="Sheet2")
df

,search_string,start_time,end_time,sentence,video_id,video_url
0,bana da,45,46,babam abime de bana da iki kardeşiz şunu söyle...,CepFJrGCWzA,https://www.youtube.com/watch?v=CepFJrGCWzA&t=45s
1,bana da,211,213,bana da bol bol şans getir tamam mı çiçeğim se...,jcqVzKEV9kI,https://www.youtube.com/watch?v=jcqVzKEV9kI&t=...
2,bana da,369,371,da devriye yapıyormuş ormancılar zaten dolaşıy...,UIJMtBZOhRg,https://www.youtube.com/watch?v=UIJMtBZOhRg&t=...
3,bana da,6696,6698,bana da silah ver,MvEFmdxPhDo,https://www.youtube.com/watch?v=MvEFmdxPhDo&t=...
4,bana da,4692,4694,allah bana da nasip eder inşallah,BhkOHUvTt7s,https://www.youtube.com/watch?v=BhkOHUvTt7s&t=...
...,...,...,...,...,...,...
165,var gibi,184,186,böyle hafif bir rengi var gibi gözüküyor,IW4n2NXyZn0,https://www.youtube.com/watch?v=IW4n2NXyZn0&t=...
166,var mı,3715,3716,annesiyle veya babasıyla başına komik bir şey ...,7pgJFwOBqaQ,https://www.youtube.com/watch?v=7pgJFwOBqaQ&t=...
167,var mı,365,367,canlandırıcı etkisi var mı bilmiyorum ama cild...,yxMKcWVOCq0,https://www.youtube.com/watch?v=yxMKcWVOCq0&t=...
168,var mı,805,806,yazıyor nasıl oldu ben gelir gelmez baktım zat...,OCbeS5MVDXg,https://www.youtube.com/watch?v=OCbeS5MVDXg&t=...


In [3]:
df_count = df.groupby(["search_string"])[["search_string"]].count()
df_count.rename(columns={"search_string":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,search_string,count
0,bu çok,8
1,bir çok,8
2,değil mi,8
3,daha çok,8
4,ben de,7
5,o da,7
6,ne gibi,7
7,bu da,7
8,bu değil,7
9,bana da,6


In [4]:
df_count_merge = pd.merge(df, df_count, how="inner",on= "search_string")
df_count_merge.sort_values(by=["count","search_string"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,search_string,start_time,end_time,sentence,video_id,video_url,count
0,değil mi,2402,2404,günah derecesi varsa o bir rütbe değil mi,CI1b3qzSmAk,https://www.youtube.com/watch?v=CI1b3qzSmAk&t=...,8
1,değil mi,5086,5088,aklına gelmişmiş sanki hiç aklından çıktı o bi...,MWMmHcPKvfs,https://www.youtube.com/watch?v=MWMmHcPKvfs&t=...,8
2,değil mi,6390,6391,zümrüt peruğu unutmadın değil mi,UvF7rFqqfmc,https://www.youtube.com/watch?v=UvF7rFqqfmc&t=...,8
3,değil mi,573,575,hazırız değil mi devran,I38DAHs79gc,https://www.youtube.com/watch?v=I38DAHs79gc&t=...,8
4,değil mi,19,21,niye parasıyla değil mi bak bir haftalık da pe...,2D4JD2O9X3w,https://www.youtube.com/watch?v=2D4JD2O9X3w&t=19s,8
...,...,...,...,...,...,...,...
165,bu kadar,5020,5022,bu kadar tanıdın buna şaşıyorum,9dhhuPr0JyI,https://www.youtube.com/watch?v=9dhhuPr0JyI&t=...,3
166,bu kadar,264,268,şimdilik bu kadar yeterli kesme tahtasına alıy...,RYc_1EmlETw,https://www.youtube.com/watch?v=RYc_1EmlETw&t=...,3
167,bu kadar,6024,6026,i nşallah teftiş esnasında da bu kadar eğlenirsin,eFKv8712V4w,https://www.youtube.com/watch?v=eFKv8712V4w&t=...,3
168,ne için,5131,5132,de bu neyin adağıydı ne için dua edeceğiz biz ...,eJdWTp7DDf0,https://www.youtube.com/watch?v=eJdWTp7DDf0&t=...,2


In [ ]:
df_count_merge["search_string"].nunique()

In [ ]:
df_count_merge[df_count_merge["count"] == 1]

In [5]:
df_count_merge.to_excel("Twogram Threegram Youtube Link Selected_Count.xlsx", index=False)

In [ ]:
df_word_group_time_loc_manuel = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Count_Manuel.xlsx")
df_word_group_time_loc_manuel.sort_values(by=["count","search_string"],inplace=True, ascending=False)
df_word_group_time_loc_manuel.reset_index(drop=True, inplace=True)
df_word_group_time_loc_manuel

In [ ]:
df_word_group_time_loc_manuel.to_excel("Twogram Threegram Youtube Link Selected_Manuel.xlsx", index=False)